# Importing required libraries

In [2]:
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Flatten,Dropout,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [5]:
color_mode="grayscale"

In [6]:
train_dir="train"
val_dir="test"
train_datagen=ImageDataGenerator(rescale=1./255)
val_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

val_generator=val_datagen.flow_from_directory(
    val_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [7]:
model=Sequential([
    Conv2D(32,(3,3),activation="relu",input_shape=(48,48,1)),
    MaxPooling2D((2,2)),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D((2,2)),
    Dropout(0.25),
    Conv2D(128,(3,3),activation="relu"),
    MaxPooling2D((2,2)),
    Conv2D(128,(3,3),activation="relu"),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation="relu"),
    Dropout(0.5),
    Dense(7,activation="softmax")
])

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 10, 10, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         7

In [14]:
model.fit(train_generator,validation_data=val_generator,epochs=25)

Epoch 1/25
449/449 [==============================] - 638s 1s/step - loss: 1.0519 - accuracy: 0.5995 - val_loss: 1.1178 - val_accuracy: 0.5808
Epoch 2/25
449/449 [==============================] - 82s 183ms/step - loss: 1.0279 - accuracy: 0.6133 - val_loss: 1.0965 - val_accuracy: 0.5869
Epoch 3/25
449/449 [==============================] - 53s 117ms/step - loss: 1.0057 - accuracy: 0.6183 - val_loss: 1.1112 - val_accuracy: 0.5816
Epoch 4/25
449/449 [==============================] - 53s 119ms/step - loss: 0.9845 - accuracy: 0.6252 - val_loss: 1.0977 - val_accuracy: 0.5943
Epoch 5/25
449/449 [==============================] - 50s 112ms/step - loss: 0.9615 - accuracy: 0.6368 - val_loss: 1.1062 - val_accuracy: 0.5896
Epoch 6/25
449/449 [==============================] - 50s 112ms/step - loss: 0.9421 - accuracy: 0.6433 - val_loss: 1.1055 - val_accuracy: 0.5871
Epoch 7/25
449/449 [==============================] - 49s 110ms/step - loss: 0.9257 - accuracy: 0.6500 - val_loss: 1.1050 - val_accu

# saving the model

In [11]:
model.save("my_model.h5")

# Real-Time Testing

In [12]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [13]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

cap=cv2.VideoCapture(0)

while True:
    ret,frame=cap.read(0)
    if not ret:
        break
    gr_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gr_frame,scaleFactor=1.3,minNeighbors=5)
    print(faces)

    for(x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray = gr_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        results = model.predict(cropped_img)
        maxindex = int(np.argmax(results))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow("Detection",cv2.resize(frame,(500,500)))
        

    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

    

()
[[230 200 196 196]]
1/1 [==============================] - 0s 231ms/step
[[228 197 203 203]]
1/1 [==============================] - 0s 23ms/step
[[228 197 203 203]]
1/1 [==============================] - 0s 22ms/step
[[228 197 203 203]]
1/1 [==============================] - 0s 22ms/step
[[227 194 206 206]]
1/1 [==============================] - 0s 21ms/step
[[231 198 201 201]]
1/1 [==============================] - 0s 26ms/step
[[231 196 202 202]]
1/1 [==============================] - 0s 22ms/step
[[230 197 202 202]]
1/1 [==============================] - 0s 24ms/step
[[231 197 201 201]]
1/1 [==============================] - 0s 25ms/step
[[230 196 201 201]]
1/1 [==============================] - 0s 24ms/step
[[233 198 196 196]]
1/1 [==============================] - 0s 25ms/step
[[236 197 196 196]]
1/1 [==============================] - 0s 26ms/step
[[238 200 196 196]]
1/1 [==============================] - 0s 25ms/step
[[235 198 196 196]]
1/1 [==============================] - 0